# Applied Data Science Capstone Project

#### Opening a New Shopping Center in Mumbai, (Maharashtra, India)

- Build a dataframe of neighborhoods in Mumbai, Maharashtra by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping Center

#### 1. Import libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


#### 2. Scrap data from Wikipedia page into a DataFrame

In [4]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Mumbai").text

In [5]:
soup = BeautifulSoup(data, 'html.parser')

In [6]:
neighborhoodList = []

In [7]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# Mumbai is 'king of land' hence named "kl"
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,List of neighbourhoods in Mumbai
1,Aarey Milk Colony
2,Agripada
3,Altamount Road
4,"Amboli, Mumbai"


In [9]:
kl_df.shape

(137, 1)

#### 3. Get the geographical coordinates

In [12]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, mumbai, Maharashtra'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [14]:
coords

[[19.19591100000001, 72.87831450000002],
 [19.166115394396662, 72.85940270774424],
 [18.976260000000025, 72.82615000000004],
 [18.96433788604836, 72.80783688604835],
 [18.940170000000023, 72.83486000000005],
 [19.14510000000007, 72.84675000000004],
 [19.026350000000036, 72.86634000000004],
 [19.042830000000038, 72.92734000000007],
 [19.095240000000047, 72.89255000000003],
 [18.920250335276123, 72.82611517568816],
 [19.06293000000005, 72.92666000000008],
 [18.935410000000047, 72.83981000000006],
 [19.054220000000043, 72.84019000000006],
 [19.062070000000062, 72.86849000000007],
 [19.16784000000007, 72.83293000000003],
 [18.95130000000006, 72.82930000000005],
 [19.140256823437614, 72.8047790882812],
 [18.969030000000032, 72.80384000000004],
 [18.980740000000026, 72.84075000000007],
 [19.138910000000067, 72.93818000000005],
 [18.9497029367247, 72.82732610600179],
 [18.996790000000033, 72.84654000000006],
 [19.110160000000064, 72.89174000000008],
 [18.985940000000028, 72.83117000000004],
 

In [15]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [16]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [17]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(137, 3)


,Neighborhood,Latitude,Longitude
0,List of neighbourhoods in Mumbai,19.195911,72.878315
1,Aarey Milk Colony,19.166115,72.859403
2,Agripada,18.976260,72.826150
3,Altamount Road,18.964338,72.807837
4,"Amboli, Mumbai",18.940170,72.834860
5,Amrut Nagar,19.145100,72.846750
6,Antop Hill,19.026350,72.866340
7,Anushakti Nagar,19.042830,72.927340
8,Asalfa,19.095240,72.892550
9,Badhwar Park,18.920250,72.826115


In [19]:
kl_df.to_csv("kl_df.csv", index=False)

#### 4. Create a map of mumbai with neighborhoods superimposed on top

In [20]:
# get the coordinates of Kuala Lumpur
address = 'mumbai, Maharashtra'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of mumbai, Maharashtra {}, {}.'.format(latitude, longitude))

The geograpical coordinate of mumbai, Maharashtra 18.9387711, 72.8353355.


In [21]:
# create map of Mumbai using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [22]:
# save the map as HTML file
map_kl.save('map_kl.html')

#### 5. Use the Foursquare API to explore the neighborhoods

In [23]:
# define Foursquare Credentials and Version
CLIENT_ID = '22A4IENI2NMUPJG2STPHO3BGJ5VYT31CMTEQ30BIMH5O4F2M' # your Foursquare ID
CLIENT_SECRET = 'PNXA02XDWOP0FR14QCIK5BY405UKROVGOFG0NWJLTOH2JU1U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 22A4IENI2NMUPJG2STPHO3BGJ5VYT31CMTEQ30BIMH5O4F2M
CLIENT_SECRET:PNXA02XDWOP0FR14QCIK5BY405UKROVGOFG0NWJLTOH2JU1U


#### Now, let's get the top 50 venues that are within a radius of 2000 meters

In [26]:
radius = 2000
LIMIT = 50

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [27]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6312, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,List of neighbourhoods in Mumbai,19.195911,72.878315,Pizza Hut,19.208620,72.872603,Pizza Place
1,List of neighbourhoods in Mumbai,19.195911,72.878315,Gold's Gym,19.213234,72.874143,Gym
2,List of neighbourhoods in Mumbai,19.195911,72.878315,Thakur Village,19.210388,72.873716,Neighborhood
3,List of neighbourhoods in Mumbai,19.195911,72.878315,McDonald's,19.208889,72.872890,Fast Food Restaurant
4,List of neighbourhoods in Mumbai,19.195911,72.878315,Ambience the rest. And bar,19.206413,72.865262,Restaurant


#### Let's check how many venues were returned for each neighorhood

In [28]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Aarey Milk Colony,50,50,50,50,50,50
Agripada,50,50,50,50,50,50
Altamount Road,50,50,50,50,50,50
"Amboli, Mumbai",50,50,50,50,50,50
Amrut Nagar,50,50,50,50,50,50
Antop Hill,50,50,50,50,50,50
Anushakti Nagar,16,16,16,16,16,16
Asalfa,50,50,50,50,50,50
Badhwar Park,50,50,50,50,50,50


#### Let's find out how many unique categories can be curated from all the returned venues

In [29]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 193 uniques categories.


In [30]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Pizza Place', 'Gym', 'Neighborhood', 'Fast Food Restaurant',
       'Restaurant', 'Indian Restaurant', 'Coffee Shop', 'Sandwich Place',
       'Ice Cream Shop', 'Nightclub', 'Department Store',
       'General Entertainment', 'Dessert Shop', 'Stadium', 'Food Truck',
       'Gym / Fitness Center', 'Bakery', 'Indie Movie Theater',
       'Snack Place', 'Udupi Restaurant', 'Movie Theater',
       'Residential Building (Apartment / Condo)', 'Flower Shop',
       'Pharmacy', 'Playground', 'Steakhouse', 'Hotel', 'Multiplex',
       'Brewery', 'Donut Shop', 'Shopping Mall', 'Hotel Bar',
       'Breakfast Spot', 'Seafood Restaurant',
       'Vegetarian / Vegan Restaurant', 'Italian Restaurant', 'Bar',
       'Bookstore', 'Asian Restaurant', 'Electronics Store',
       'Design Studio', 'Clothing Store', 'Chinese Restaurant', 'Bistro',
       'Burger Joint', 'Farm', 'Café', 'Club House', 'Golf Course',
       'Middle Eastern Restaurant'], dtype=object)

In [31]:
# check if the results contain "Shopping Center"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

#### 6. Analyze Each Neighborhood

In [32]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6312, 194)


,Neighborhoods,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Water Park,Women's Store,Zoo
0,List of neighbourhoods in Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,List of neighbourhoods in Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,List of neighbourhoods in Mumbai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(137, 194)


,Neighborhoods,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Water Park,Women's Store,Zoo
0,Aarey Milk Colony,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.020000,0.00,0.00,0.000000,0.00,0.020000,0.060000,0.000000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.040000,0.000000,0.00,0.00,0.020000,0.020000,0.00,0.020000,0.00,0.0000,0.000000,0.020000,0.00,0.00,0.020000,0.020000,0.00,0.00,0.040000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.02,0.000000,0.00,0.000000,0.040000,0.00,0.020000,0.000000,0.00,0.020000,0.00,0.120000,0.000000,0.000000,0.00,0.00000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00000,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.020000,0.02000,0.00,0.000000,0.00,0.000000,0.00000,0.0000,0.020000,0.020000,0.000,0.000000,0.00,0.120000,0.00000,0.00,0.00,0.020000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.00,0.00,0.040000,0.00,0.000000,0.00000,0.00,0.00000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.040000,0.00,0.0000,0.00000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.00,0.040000,0.00,0.020000,0.020000,0.000000,0.00,0.020000,0.00,0.00,0.000000,0.000000,0.

In [34]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

20

#### Create a new DataFrame for Shopping Mall data only

In [35]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [36]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Aarey Milk Colony,0.02
1,Agripada,0.02
2,Altamount Road,0.00
3,"Amboli, Mumbai",0.00
4,Amrut Nagar,0.00


#### 7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Mumbai into 3 clusters.

In [37]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 0])

In [38]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [39]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Aarey Milk Colony,0.02,1
1,Agripada,0.02,1
2,Altamount Road,0.00,0
3,"Amboli, Mumbai",0.00,0
4,Amrut Nagar,0.00,0


In [40]:
# merge mumbai_grouped with mumbai_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(137, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.02,1,19.166115,72.859403
1,Agripada,0.02,1,18.976260,72.826150
2,Altamount Road,0.00,0,18.964338,72.807837
3,"Amboli, Mumbai",0.00,0,18.940170,72.834860
4,Amrut Nagar,0.00,0,19.145100,72.846750


In [41]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(137, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
68,Khotachiwadi,0.000000,0,18.954850,72.820430
96,Nala Sopara,0.000000,0,19.419990,72.814030
95,Nagpada,0.000000,0,18.968620,72.833020
94,Mira Road,0.000000,0,19.126406,72.839544
93,Mendham's Point,0.000000,0,18.940170,72.834860
92,Mazagaon,0.000000,0,18.968360,72.841750
91,"Matunga Road, Mumbai",0.000000,0,19.029210,72.846930
90,Marol,0.000000,0,19.119140,72.882820
89,Marine Lines,0.000000,0,18.943430,72.823190
88,"Marine Drive, Mumbai",0.000000,0,18.946660,72.824560


#### Finally, let's visualize the resulting clusters

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### 8. Examine Clusters

#### Cluster 0

In [44]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
68,Khotachiwadi,0.0,0,18.954850,72.820430
96,Nala Sopara,0.0,0,19.419990,72.814030
95,Nagpada,0.0,0,18.968620,72.833020
94,Mira Road,0.0,0,19.126406,72.839544
93,Mendham's Point,0.0,0,18.940170,72.834860
92,Mazagaon,0.0,0,18.968360,72.841750
91,"Matunga Road, Mumbai",0.0,0,19.029210,72.846930
90,Marol,0.0,0,19.119140,72.882820
89,Marine Lines,0.0,0,18.943430,72.823190
88,"Marine Drive, Mumbai",0.0,0,18.946660,72.824560


#### Cluster 1

In [45]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
1,Agripada,0.020000,1,18.976260,72.826150
134,Western Suburbs (Mumbai),0.020000,1,19.197000,72.827630
7,Asalfa,0.020000,1,19.095240,72.892550
0,Aarey Milk Colony,0.020000,1,19.166115,72.859403
50,Gokuldham,0.023256,1,19.172270,72.869670
36,Dava Bazaar,0.021277,1,19.131400,72.926960
108,Poisar,0.020000,1,19.211530,72.852680
28,Cotton Green,0.020000,1,18.985380,72.841210
102,Orlem,0.020000,1,19.195740,72.835310
99,Navy Nagar,0.031250,1,18.906460,72.814930


#### Cluster 2

In [46]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
13,Bangur Nagar,0.040000,2,19.16784,72.832930
72,Lalbaug,0.040000,2,18.99294,72.838370
23,Chinchpokli,0.040000,2,18.98594,72.831170
106,Parel,0.040000,2,18.99566,72.839070
85,Malad,0.060000,2,19.18655,72.848360
135,Yashodham,0.040000,2,19.19021,72.853661
60,Kajuwadi,0.040000,2,19.19822,72.961720
31,Currey Road railway station,0.040000,2,18.99515,72.832730
18,C.G.S. colony,0.047619,2,19.13891,72.938180


#### Observations:

Most of the shopping Centers are concentrated in the central area of Mumbai city, with the highest number in cluster 0 and moderate number in cluster 1. On the other hand, cluster 2 has very low number to totally no shopping Center in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping Center as there is very little to no competition from existing Shopping centers. Meanwhile, shopping Centers in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of shops. From another perspective, this also shows that the oversupply of shopping centers mostly happened in the central area of the city, with the suburb area still have very few shopping centers. Therefore, this project recommends property developers to capitalize on these findings to open new shopping centers in neighborhoods in cluster 2 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping centers in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 which already have high concentration of shopping centres and suffering from intense competition.